In [1]:
# import dependencies
import pandas as pd
import numpy as np
import random


In [2]:
# function to determine new threshold case count
def nthThresh(n,t):
    return 100 * n + t


In [3]:
# function to adjust infectProbs weights and calculate r0 for corresponding threshold
def thisThresh():
    if n == 1:
        infectProbs[3] -= 1
        infectProbs[2] += 1
    elif n == 2:
        infectProbs[3] -= 1
        infectProbs[1] += 1
    elif n == 13:
        infectProbs[2] -= 1
        infectProbs[0] += 1
    elif n == 14:
        infectProbs[1] -= 1
        infectProbs[0] += 1
    elif n != 0:
        infectProbs[3] -= 1
        infectProbs[0] += 1
    r0 = (infectProbs[1] + 2*infectProbs[2] + 3*infectProbs[3]) / sum(infectProbs)

In [4]:
# function to calculate ranges for probability distribution if statements
def ranges(probsArray):
    rangesArray = []
    for i in range(0, len(probsArray)):
        rangesArray.append(sum(probsArray[0:i+1]))
    return rangesArray


In [5]:
# function to generate a value based on a probability distribution array
def decide(probsArray):
    rangesArray = ranges(probsArray)
    ran = random.randint(0,sum (probsArray)-1)
    for i in range(0, len(rangesArray)):
        if ran < rangesArray[i]:
            return i


In [6]:
# probabilities of each demographic being selected
demoProbs = [1,1,2,2,3,1]
# weighted probabilities for #people the current case can infect
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
infectProbs = [0,0,0,12]
# cases remaining (one case to start it off)
more = 1
# cases that have occured
casecount = 0
# deaths that have occured
deathcount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# dictionary with all data
weathering = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0
}


In [7]:
while more > 0:
    # remove current case from how many more cases left
    more -= 1
    # add current case to total # of cases
    casecount += 1

    # if case count is at or over threshold number
    if casecount >= nthThresh(n, thresh):
        # add one to threshold n value
        n += 1
        # calculate new threshold
        thresh = nthThresh(n, thresh)
        # adjust infectProbs weights and calculate r0 for corresponding threshold
        thisThresh()

    # decide how many people the current case infects and add them to more
    more += decide(infectProbs)
    
    weathering[decide(demoProbs)] += 1

weathering
    
    

{0: 1118, 1: 1173, 2: 2286, 3: 2320, 4: 3505, 5: 1138}

In [8]:
for i in range(0, len(weathering)):
    print(100*(weathering[i]/casecount))
    

9.688041594454074
10.164644714038127
19.80935875216638
20.103986135181977
30.37261698440208
9.861351819757367
